In [1]:
# k crossover d，close > ema 8, ema 8 > ema 14, ema 14 > 50, take profit 2 atr, stop loss 3 atr

In [2]:
import websocket
import requests
import json
import pandas as pd
import datetime as dt
import numpy as np
import time
import ta
from dotenv import load_dotenv
import os
from binance.client import Client
from IPython.display import clear_output
import winsound

In [3]:
frequency = 250  # Set Frequency To 2500 Hertz
duration = 300  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

In [4]:
load_dotenv()

api_key = os.getenv('API_KEY')
api_secret = os.getenv('SECRET_KEY')

client = Client(api_key, api_secret, testnet = False)

spot = client.get_account()
s_bal = pd.DataFrame(spot['balances'])
print(s_bal)


    asset        free      locked
0     BTC  0.00000000  0.00000000
1     LTC  0.00000000  0.00000000
2     ETH  0.00000000  0.00000000
3     NEO  0.00000000  0.00000000
4     BNB  0.24622450  0.00000000
..    ...         ...         ...
529   GNS  0.00000000  0.00000000
530   SYN  0.00000000  0.00000000
531  LQTY  0.00000000  0.00000000
532    ID  0.00000000  0.00000000
533   ARB  0.00000000  0.00000000

[534 rows x 3 columns]


In [5]:
timezone = 8
endpoint = 'wss://stream.binance.com:9443/ws'
symbol = 'ethusdt'
symbol_C = symbol.upper()
interval = '15m'

# start epoch till now
start_time = 1672586510000
end_time = round(time.time() * 1000)

# step between timestamps in milliseconds, 60000 = 1min 
step = 60000 * 36000

In [6]:
data = json.dumps({'method':'SUBSCRIBE','params':[symbol + '@kline_' + interval],'id':1})

In [7]:
# 先抓歷史資料好讓技術指標能成型
def get_historical(symbol, interval, start_time, end_time, step):
    
    rawdf = pd.DataFrame()
    
    url = "https://api.binance.com/api/v3/klines"
    
    for timestamp in range(start_time, end_time, step):
        params = {"symbol": symbol_C,
                  "interval": interval,
                  "startTime": timestamp,
                  "endTime": timestamp + step}
        response = requests.get(url, params=params).json()
        out = pd.DataFrame(response, columns = ["Open time", "Open", "High", "Low", "Close",
                                               "Volume", "Close_Time", "Quote asset volume",
                                               "Number of trades", "Taker buy base asset volume",
                                               "Taker buy quote asset volume", "Ignore"])
        rawdf = pd.concat([rawdf, out], axis = 0)
    
    rawdf = rawdf[['Close_Time', 'Open', 'Close', "High", "Low", 'Volume']]
    convert_dict = {'Close_Time': float, 'Open': float, 'Close': float, "High": float, "Low": float, 'Volume': float}
    rawdf = rawdf.astype(convert_dict)

    rawdf['Close_Time'] = pd.to_datetime(rawdf['Close_Time'], unit = 'ms')
    rawdf['Close_Time'] = rawdf['Close_Time'] + pd.Timedelta(hours=timezone)
    rawdf['Close_Time'] = rawdf['Close_Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    rawdf = rawdf.reset_index(drop=True)
    
    return rawdf

In [8]:
rawdf = get_historical(symbol, interval, start_time, end_time, step)
print(rawdf)

               Close_Time     Open    Close     High      Low      Volume
0     2023-01-01 23:44:59  1195.21  1196.08  1196.14  1195.21    736.9041
1     2023-01-01 23:59:59  1196.08  1196.81  1196.81  1195.60    573.3049
2     2023-01-02 00:14:59  1196.81  1196.07  1197.74  1195.88   1242.7586
3     2023-01-02 00:29:59  1196.07  1197.70  1197.71  1196.06   3257.3197
4     2023-01-02 00:44:59  1197.71  1196.88  1198.35  1196.60   3277.6349
...                   ...      ...      ...      ...      ...         ...
1995  2023-03-23 03:29:59  1746.04  1752.64  1757.93  1740.00  22164.5787
1996  2023-03-23 03:44:59  1752.65  1730.54  1753.12  1730.00  24721.7164
1997  2023-03-23 03:59:59  1730.54  1716.84  1733.15  1710.12  36469.2629
1998  2023-03-23 04:14:59  1716.84  1719.02  1726.94  1710.30  23097.5465
1999  2023-03-23 04:29:59  1719.02  1722.05  1729.83  1716.78  17601.1406

[2000 rows x 6 columns]


In [9]:
def indicators(df):

# ema
    for i in (8, 14, 50):
        df['ema_'+str(i)] = ta.trend.ema_indicator(df.Close, window=i)

# atr
    df['atr'] = ta.volatility.average_true_range(df.High, df.Low, df.Close)

# kd
    kd_int = 14
    d_int = 3
   
    kddf = pd.DataFrame()
    kddf[str(kd_int) + '-Low'] = df['Low'].rolling(kd_int).min()
    kddf[str(kd_int) + '-High'] = df['High'].rolling(kd_int).max()
    df['slowk'] = (df['Close'] - kddf[str(kd_int) + '-Low'])*100/(kddf[str(kd_int) + '-High'] - kddf[str(kd_int) + '-Low'])
    df['slowd'] = df['slowk'].rolling(d_int).mean()
    
    return df

def checkcross(df):
    series = df['slowk'] > df['slowd']
    return series.diff() & series

In [10]:
def conditions(df):
    # k crossover d，close > ema 8, ema 8 > ema 14, ema 14 > 50, take profit 2 atr, stop loss 3 atr
    df['kd_cross'] = checkcross(df)

    for index, row in df.iterrows():
        # c1
        df['c1'] = df['kd_cross']
        # c2
        df['c2'] = df['Close'] <= df['ema_8']
        # c3
        df['c3'] = df['ema_8'] <= df['ema_14']
        # c4
        df['c4'] = df['ema_14'] <= df['ema_50']

    # 條件達成
    df['signal'] = False
    df.loc[df.c1 & df.c2 & df.c3 & df.c4, 'signal'] = True
            
    # 下一根進場
    df['open_entry'] = False

    return df

In [11]:
def enter_position(df):
    global symbol_C
    
    #-----Calculate entry price-----#
    close_val = df['Close']
    atr_val = df['atr']
    df.loc[df.index[-1], 'entry_p'] = close_val.loc[close_val.index[-2]]
    df.loc[df.index[-1], 'stop_loss'] = close_val.loc[close_val.index[-2]] + 3.2 * atr_val.loc[atr_val.index[-2]]
    df.loc[df.index[-1], 'take_profit'] = close_val.loc[close_val.index[-2]] - 3.5 * atr_val.loc[atr_val.index[-2]]

    #-----position attributes-----#
    quantity = 0.01
    entry_p = round(df.loc[df.index[-2], 'Close'],2)
    stop_loss_p = round(df.loc[df.index[-1], 'stop_loss'],2)
    take_profit_p = round(df.loc[df.index[-1], 'take_profit'],2)

    print(df.loc[df.index[-1], 'entry_p'] == entry_p)  
    
    try:
        order = client.futures_create_order(
            symbol=symbol_C,
            side='SELL',
            type='LIMIT',
            timeInForce='GTC',
            quantity=quantity,
            price=entry_p,
            positionSide='SHORT',
        )
        print('Order created successfully.')       
        
        stop_loss_order = client.futures_create_order(
            symbol=symbol_C,
            side='BUY',
            type='STOP_MARKET',
            stopPrice=stop_loss_p,
            stopLimitTimeInForce='GTC',
            closePosition = 'true',
            positionSide = 'SHORT',
        )
        print('Stop loss created successfully')
        
        take_profit_order = client.futures_create_order(
            symbol=symbol_C,
            side='BUY',
            type='TAKE_PROFIT_MARKET',
            stopPrice=take_profit_p,
            stopLimitTimeInForce='GTC',
            closePosition = 'true',
            positionSide = 'SHORT',
        )
        winsound.Beep(frequency, duration)
        print('Take profit created successfully')
            
    except Exception as e:
        print(f'Error creating order: {e}')

    print(str(symbol_C) + ' entered at ' + str(entry_p) + ', stop loss at ' + str(stop_loss_p))


In [12]:
line_count = 0
max_lines = 5

current_k = 0

def on_open(ws):
    ws.send(data)
    
def on_message(ws, message):
    global current_k, line_count, max_lines
    df = get_historical(symbol, interval, start_time, end_time, step)

    indicators(df)
    conditions(df)
    
    try:
        # 如果時間標籤改變，代表上個 Close 發生，檢查是否進場
        if df['Close_Time'][len(df) - 1] != current_k:
            print('time changed')
            current_k = df['Close_Time'][len(df) - 1]
            if df.iloc[df.index[-2]]['signal'] == True:
                winsound.Beep(frequency, duration)
                df.loc[df.index[-1], 'open_entry'] = True
                enter_position(df)
                print(df.tail(2))
        
        # 隨時偵測出場條件是否成立，不必等 Close 發生
#         if ((df.loc[df.index[-1], 'rsi'] >= tp_rsi) & (df.loc[df.index[-1], 'slowk'] >= tp_slowk) & (df.loc[df.index[-1], 'slowd'] >= tp_slowd)):
#             take_profit_order = client.futures_create_order(
#                 symbol=symbol_C,
#                 side='SELL',
#                 type='TAKE_PROFIT_MARKET',
#                 stopPrice=df.loc[df.index[-1], 'Close'],
#                 stopLimitTimeInForce='GTC',
#                 closePosition = 'true',
#                 positionSide = 'LONG'
#             )
#             winsound.Beep(frequency, duration)
#             print('Take profit created successfully')
        
    except Exception as e:
        print(f'Error creating order: {e}')


    try:
        df = df.reset_index(drop=True)
        print('nothing happened...' + str(df.loc[df.index[-1], 'Close_Time']) + ' ' + str(df.loc[df.index[-1], 'Close']))
        print()
        print(df.tail(2))
        print('----------------------------------------------------------------------------')
        
        positions = client.futures_account()['positions']
        for position in positions:
            if float(position['positionAmt']) != 0:
                position_df = pd.DataFrame({'Symbol':position['symbol'],
                                            'Entry_P':round(float(position['entryPrice']),2),
                                            'Amt':round(float(position['positionAmt']) * df.loc[df.index[-1], 'Close'],2),
                                            'PL':round(float(position['unrealizedProfit']),2),
                                            'X':round(float(position['leverage']),1),
                                           }, index=[0])      
                print(position_df)
    
    except Exception as e:
        print(f'Error: {e}')    
        
        
    line_count += 1
    if line_count >= max_lines:
        clear_output(wait=True)
        line_count = 0

In [13]:
ws = websocket.WebSocketApp(endpoint, on_message = on_message, on_open = on_open)
ws.run_forever()

time changed
True
Order created successfully.
Stop loss created successfully
Error creating order: APIError(code=-4015): Client order id is not valid.
ETHUSDT entered at 1719.02, stop loss at 1776.6
               Close_Time     Open    Close     High      Low      Volume  \
1998  2023-03-23 04:14:59  1716.84  1719.02  1726.94  1710.30  23097.5465   
1999  2023-03-23 04:29:59  1719.02  1722.05  1729.83  1716.78  17601.1406   

            ema_8       ema_14       ema_50        atr  ...  kd_cross     c1  \
1998  1744.397539  1758.934609  1782.621862  17.992622  ...      True   True   
1999  1739.431419  1754.016661  1780.246495  17.639578  ...     False  False   

        c2    c3    c4  signal  open_entry  entry_p    stop_loss  take_profit  
1998  True  True  True    True       False      NaN          NaN          NaN  
1999  True  True  True   False        True  1719.02  1776.596391  1656.045822  

[2 rows x 22 columns]
nothing happened...2023-03-23 04:29:59 1722.05

               Cl

False